# Setting up a Georeferenced Road Accident Database in Senegal: Automatic Data Extraction and GIS Integration

### FIRST PART: Data Collection from SENENEWS

In [1]:
import pandas as pd # pour le database
import requests    # pour envoyer une requete 
from bs4 import BeautifulSoup   # pour la collection
from text_to_num import alpha2digit  # pour remplacer les nombres ecrivent en lettres en chiffre 
import spacy # pour le traitement de texte en decoupant le texte en token 
import re  # pour supprimer les espaces 

In [2]:
# Scraper les communes, les arrondissements et les villages du Senegal

url_ = "https://www.planete-senegal.com/senegal/decoupage_administratif_senegal.php"

reponse = requests.get(url_)

if reponse.status_code == 200:
    #recuperer le code html
    html = reponse.text
    
    with open("Departement_Senegal.html", "w", encoding="utf-8") as f:
            f.write(html)
    
    communes, arrondissements, villages = [], [], []
     
    
    # scraper
    soup = BeautifulSoup(html, "lxml")
    
    # Scraper tous les communes, les arrondissements et les villages du Senegal
    all_content_page = soup.find("table", width = "100%", border = "1")
    all_communes = all_content_page.find_all("td", width = "20%")
    all_arrondissement = all_content_page.find_all("td", width = "26%")
    all_villages = all_content_page.find_all("td", width = "39%")
    
    # Les communes
    for commune in all_communes:
        communes.append(commune.text.strip().replace("\r\n", ""))
        
    # Les arrondissements
    for arrondissement in all_arrondissement:
        arrondissements.append(arrondissement.text.strip().replace("\r\n", ""))
        
    # Les villages
    for village in all_villages:
        villages.append(village.text.strip().replace("\r\n", ""))
        
    # Remplacer les espaces multiples par un seul espace
    resultat_communes_propres = [re.sub(r'\s+', ' ', commune) for commune in communes]
    resultat_arrondissement_propres = [re.sub(r'\s+', ' ', arrondissement) for arrondissement in arrondissements]
    resultat_villages_propres = [re.sub(r'\s+', ' ', village) for village in villages]
    
    
    # Separer tous les communes en supprimant le caractere "-"
    communes_propres = []
    for com in resultat_communes_propres:
        if com != "Rosso-Sénégal":
            communes_propres.extend(com.strip().split("-"))
            
           
    # Separer tous les arrondissements en supprimant le caractere "-"
    arrondissements_propres = []
    for arron in resultat_arrondissement_propres:
        if arron not in ["Dakar-Plateau", "Ross-Béthio", "Cas-cas"]:
            arrondissements_propres.extend(arron.strip().split("-"))
            
            
    # Separer tous les villages en supprimant le caractere "-"
    villages_propres = []
    for villa in resultat_villages_propres:
        villages_propres.extend(villa.strip().split("-"))
    
    # suppresion des cases vides    
    villages_propres = list(filter(None, villages_propres))
     
     
    # Regroupement communes, les arrondissements et les villages du Senegal
    senegal = set(communes_propres) | set(arrondissements_propres) | set(villages_propres)
    senegal = list(senegal)
    
    # suppresion des espaces vides
    senegal = [n.strip() for n in senegal]
    
    # Enrichir la liste senegal
    senegal.extend(("Niaro", "Guédiawaye"))
    
    print(senegal)
    
else:
    print("ERREUR:", reponse.status_code)

['Keur Maba Diakhou', 'Mbane', 'Matam', 'Sagatta Dioloff', 'Gainte Pathé', 'Birkelane', 'Pata', 'Pout', 'Paoscoto', 'Dangalma', 'Kab Gaye', 'Yop', 'Keur Socé', 'Réfane', 'Pété Ouarack', 'Gathiary', 'Dinguiraye', 'Kaffrine', 'Thiamène Djolof', 'Santhiaba Manjacque', 'Dougue', 'Dankh Sène', 'Kahi', 'Loul Séssène', 'Taïba Niassène', 'Fissel', 'Pire Gourèye', 'Keur Momar Sarr', 'Ngabou Dalla', 'Diawara', 'Gainthe Kaye', 'Dioulacolon', 'Sagatta Dioloff', 'Ngayokhème', 'Barkedji', 'Ndiaganiao', 'Bandafassi', 'Patar', 'Nguidillé', 'Mbour', 'Gamadji Sarré', 'Ida Mouride', 'Foundiougne', 'Tomboronkoto', 'Diattacounda', 'Labgar', 'Wack Ngouna', 'Taïf', 'Ogo', 'Gaé', 'Tenghory', 'Waoundé', 'Kandia', 'Lambaye', 'Civol', 'Communautés rurales', 'Léona', 'Séssène', 'Djibabouya', 'Ourossogui', 'Fafacourou', 'Oulampane', 'Kahone', 'Ndioumane Taiba', 'Dialacoto', 'Aéré', 'Médina Yoy Foulah', 'Missirah', 'Tassette', 'Saly Escale', 'Sakar', 'Gniby', 'Fadiouth', 'Suelle', 'Kounkané', 'Ndramé Escale', 'Gran

In [3]:
# Charger le modèle spaCy
nlp = spacy.load("fr_core_news_md")

# Constantes
URL = "https://www.senenews.com/tag/accident"


HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0"
}


In [4]:
def fetch_html(url, headers):
    """Recuper le contenu HTML de l'url."""
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print("ERREUR:", response.status_code)
        return None
    


In [5]:
def save_html_to_file(html, filename):
    """Sauvarge le code HTML dans un fichier."""
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html)

In [6]:
def parse_html(html):
    """Parse contenu HTML en utilisant BeautifulSoup."""
    soup = BeautifulSoup(html, "lxml")
    return soup

In [7]:
def determine_gravity(nombre_morts):
    """Determine la gravite de l'accident."""
    if nombre_morts > 0:
        return "grave"
    else:
        return "non grave"

In [8]:
def extract_information(doc):
    """Extraction des informations sur les textes."""
    lieu_accident = None
    nombre_morts = 0
    nombre_blesses = 0
    
    def extract_numbers(token):
        prev_token = token.nbor(-1) 
        if prev_token.like_num:
            return 1 if prev_token.text.lower() == "un" else int(prev_token.text)
        return 0

    for token in doc:
        if token.text in senegal:
            lieu_accident = token.text
        elif "mort" in token.text.lower():
            nombre_morts = extract_numbers(token)
        elif "blessé" in token.text.lower():
            nombre_blesses = extract_numbers(token)
    
    if not lieu_accident:
        lieu_accident = None
    
    return lieu_accident, nombre_morts, nombre_blesses

In [9]:
# Fonction pour récupérer la date à partir du lien de l'article
def get_article_date(article_url):
    """ Recupere la date à partir du lien"""
    response = requests.get(article_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver la balise <span class="date updated">
    date_span = soup.find('span', class_='date updated')
    
    if date_span:
        return date_span.text.strip()  # Retourner le texte contenu dans la balise
    else:
        return None  # Retourner None si la balise n'est pas trouvée 

In [10]:
# Fonction pour récupérer les titres d'articles contenant le mot "accident" et leur date associée
def get_accident_articles_with_dates(soup):
   
    articles_with_dates = []

    # Trouver tous les liens des articles
    for link in soup.find_all('a', href=True):
        article_url = link['href']
        
        # Vérifier si le titre de l'article contient le mot "accident"
        if 'accident' in link.text.lower():
            # Trouver la date associée à cet article
            date = get_article_date(article_url)
            if date:
                articles_with_dates.append({'title': link.text.strip(), 'date': date})
                
        # Ajouter un délai entre chaque requête
        #time.sleep(2)  # Attendre 2 secondes entre les requêtes            

    return articles_with_dates

In [11]:
# Fonction pour supprimer les heures au niveau de la date
def remove_time(date_str):
    """Remove time from a datetime string."""
    if ' à ' in date_str:
        return date_str.split(' à ')[0]
    return date_str

In [12]:
############## PARTIE A ADAPTER POUR CHAQUE SITE #################################################

def extract_data(soup):
    """Extract relevant data from parsed HTML."""
    data = pd.DataFrame(columns=["Jour", "Lieu", "Nombre_victimes", "Nombre_blesses", "Nombre_morts", "Gravite"])
    
     # Intégrer la récupération des titres d'articles avec leur date associée
    accident_articles_with_dates = get_accident_articles_with_dates(soup) 
    
    for article in accident_articles_with_dates:
        # Extraire les informations pertinentes pour chaque article
        texte = article['title']
        date = remove_time(article['date'])
        text = alpha2digit(texte, "fr")
        doc = nlp(text)
        lieu_accident, nombre_morts, nombre_blesses = extract_information(doc)
        gravite_accident = determine_gravity(nombre_morts)
        
        # Ajouter les informations extraites dans le dataframe
        new_row = pd.DataFrame([{
            "Jour" : date,
            "Lieu": lieu_accident,
            "Nombre_victimes": nombre_morts + nombre_blesses,
            "Nombre_blesses": nombre_blesses,
            "Nombre_morts": nombre_morts,
            "Gravite": gravite_accident
        }])
        data = pd.concat([data, new_row], ignore_index=True)
    
    # Remplacer les valeurs None par "NA" dans la colonne "Lieu"
    data["Lieu"].fillna("NA", inplace=True)
    
    # Réindexer pour commencer à 1
    data.index = range(1, len(data) + 1) 
    
            
    
    return data



In [13]:
############################# FONCTION PRINCIPALE ###################################

def main():
    html = fetch_html(URL, HEADERS)  # Entrer l'url et l'entete
    
    if html:
        save_html_to_file(html, "Accident-Senenews.html")
        soup = parse_html(html) # parse le contenu html
        data = extract_data(soup) # extraction des donnees 
        
        print(data) # Affichage de la base de donnees
        
        # Export data 
        data.to_excel("C:/Users/Mouha/OneDrive/Desktop/MEMOIRE/Code_memoire_mouhamadou_djimba_thiam/scraping_senenew_memoire/data_accident_senegal.xlsx", index=False)

if __name__ == "__main__":
    main()
    

          Jour        Lieu Nombre_victimes Nombre_blesses Nombre_morts  \
1   29/05/2024          NA               0              0            0   
2   29/05/2024    Rufisque               0              0            0   
3   27/05/2024   Birkelane               9              9            0   
4   25/05/2024       Thiès               0              0            0   
5   21/05/2024          NA               0              0            0   
6   18/05/2024  Marsassoum               0              0            0   
7   15/05/2024          NA               1              0            1   
8   13/05/2024  Guédiawaye               0              0            0   
9   13/05/2024          NA               0              0            0   
10  10/05/2024          NA               0              0            0   
11  08/05/2024          NA               0              0            0   
12  07/05/2024  Ziguinchor               0              0            0   
13  05/05/2024     Sédhiou            

### DEUXIEME PARTIE : NETTOYAGE ET TRAITEMENT DES DONNEES ###